In [1]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.4 MB/s eta 0:00:00


In [2]:
import os
import math
import time
import inspect
from dataclasses import dataclass
import tiktoken
import torch
import torch.nn as nn
from torch.nn import functional as F
from transformers import get_cosine_with_hard_restarts_schedule_with_warmup
from transformers import AdamW, get_scheduler

In [3]:
class DataLoaderLite:
    def __init__(self, file_path, B, T):
        self.B = B
        self.T = T

        # Load tokens from disk and store in memory
        with open(file_path, 'r') as f:
            text = f.read()
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f'Loaded {len(self.tokens)} tokens.')

        # State
        self.current_position = 0

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position:self.current_position + B * T + 1]
        x = (buf[:-1]).view(B, T)  # Inputs
        y = (buf[1:]).view(B, T)  # Targets

        # Advance position
        self.current_position += B * T
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.current_position = 0
        return x, y

In [ ]:
class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # self.c_proj.NANGPT_SCALE_INIT = 1
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)

        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # output projection
        y = self.c_proj(y)
        return y

In [ ]:
class MLP(nn.Module):

    def __init__(self, config):
      super().__init__()
      self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd, bias=True)
      self.gelu = nn.GELU()
      self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd, bias=True)
      # self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
      x = self.c_fc(x)
      x = self.gelu(x)
      x = self.c_proj(x)
      return x

class Block(nn.Module):

    def __init__(self, config):
      super().__init__()
      self.ln_1 = nn.LayerNorm(config.n_embd)
      self.attn = CausalSelfAttention(config)
      self.ln_2 = nn.LayerNorm(config.n_embd)
      self.dropout = nn.Dropout(config.dropout)
      self.mlp = MLP(config)

    def forward(self, x):
      residual_scale = 0.7
      x = x + residual_scale * self.dropout(self.attn(self.ln_1(x)))
      x = x + residual_scale * self.dropout(self.mlp(self.ln_2(x)))
      return x

In [4]:
import torch
import torch.nn as nn

class Block(nn.Module):
    def __init__(self, config):
        """
        Transformer Block

        Args:
            embed_dim: Dimensionality of the input embeddings
            num_heads: Number of attention heads
            ff_dim: Dimensionality of the feedforward network
            dropout: Dropout rate
        """
        super(Block, self).__init__()

        # Multi-head self-attention
        self.attention = nn.MultiheadAttention(config.n_embd, config.n_head, dropout=config.dropout, batch_first=True, bias=True)

        # Feedforward network
        self.ffn = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),  # or GELU
            nn.Linear(4 * config.n_embd, config.n_embd)
        )

        # Layer normalization
        self.norm1 = nn.LayerNorm(config.n_embd, eps=1e-6)
        self.norm2 = nn.LayerNorm(config.n_embd, eps=1e-6)

        # Dropout
        self.dropout1 = nn.Dropout(config.dropout)
        self.dropout2 = nn.Dropout(config.dropout)

    def forward(self, x, mask=None):
        """
        Forward pass for the Transformer Block.

        Args:
            x: Input tensor of shape (seq_len, batch_size, embed_dim)
            mask: Attention mask (optional)

        Returns:
            Output tensor of shape (seq_len, batch_size, embed_dim)
        """
        # Multi-head self-attention with residual connection
        attn_output, _ = self.attention(x, x, x, attn_mask=mask)
        x = x + self.dropout1(attn_output)
        x = self.norm1(x)

        # Feedforward network with residual connection
        ffn_output = self.ffn(x)
        x = x + self.dropout2(ffn_output)
        x = self.norm2(x)

        return x


In [5]:
@dataclass
class GPTConfig:
    block_size: int = 1024 # max sequence length
    vocab_size: int = 50257 # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
    n_layer: int = 12 # number of layers
    n_head: int = 8 # number of heads
    n_embd: int = 768 # embedding dimension
    dropout: float = 0.1

In [6]:
class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing
        self.transformer.wte.weight = self.lm_head.weight

        # weight initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean = 0.0, std = std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std = 0.02)

    def forward(self, idx, targets=None):
        # idx is of shape (B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        # forward the token and posisition embeddings
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device) # shape (T)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (T, n_embd)
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (B, T, n_embd)
        x = tok_emb + pos_emb
        # forward the blocks of the transformer
        for block in self.transformer.h:
            x = block(x)
        # forward the final layernorm and the classifier
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

In [7]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")

# SEED
torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

using device: cpu


In [8]:
model = GPT(GPTConfig())
model.to(device)

train_loader = DataLoaderLite('/content/input.txt', B=32, T=128)

Loaded 338025 tokens.


In [9]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

Total Parameters: 124439808


In [10]:
config = GPTConfig()

In [11]:
num_training_steps = 10000

optimizer = AdamW(
    model.parameters(),
    lr=3e-4,
    betas=(0.9, 0.95),
    eps=1e-8,
    weight_decay=0.1
)

lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=1000,
    num_training_steps=num_training_steps,
    num_cycles=4
)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
best_loss = float('inf')

In [ ]:
for step in range(num_training_steps):
  x, y = train_loader.next_batch()
  x, y = x.to(device), y.to(device)
  optimizer.zero_grad()
  logits, loss = model(x, y)

  loss.backward()
  optimizer.step()
  lr_scheduler.step()

  if step % 100 == 0:
    print(f'step{step}, loss: {loss.item()}')

    if loss < best_loss and step > 9000:
      best_loss = loss
      checkpoint = {
          'step': step,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': loss,
          'config': config,
      }
      save_dir = '/content'
      torch.save(checkpoint, f'{save_dir}/best_model.pt')
      print(f"Saved best model with loss: {loss:.6f}")

step0, loss: 10.908920288085938


In [ ]:
torch.save(checkpoint, f'/content/drive/MyDrive/Colab Notebooks/best_model.pt')
print(f"Saved best model with loss: {best_loss:.6f}")